# Video Object Detection -- Rekognition Custom Labels
This sample notebook shows demonstrates how to detect custom labels in a video with Amazon Rekognition Custom Labels and draw their corresponding bounding boxes.

### Import and install necessary packages
Let's begin by importing and installing all the necessary packages we need to make the notebook run.

In [ ]:
# Install the first time you execute the notebook
!apt-get -qq update
!apt-get -qq install ffmpeg -y 
!pip install --quiet opencv-python-headless

In [ ]:
import cv2
import boto3
import os
import glob
import time
import queue
import shutil
from IPython.display import Video
from multiprocessing import Lock, Process, Queue, current_process
rekognition = boto3.client('rekognition')

To be able to use your Amazon Rekognition Custom Labels running model, insert the arn of the project below, which you will locate in the Custom Labels console.

In [ ]:
projectVersionArn = "***" ## INSERT THE ARN OF YOUR CUSTOM LABELS PROJECT

### Helper Functions
Here are a couple of helper functions we are going to use to process our video frames and detect and draw the bounding boxes.

In [ ]:
os.mkdir("input_video")
os.mkdir("output_video")

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
def transform_bounding(frame,box):
    imgWidth, imgHeight = frame
    left = int(imgWidth * box['Left'])
    top = int(imgHeight * box['Top'])
    right = left + int(imgWidth * box['Width'])
    bottom = top + int(imgHeight * box['Height'])
    return left,top,right,bottom

def process_frames(frames_list):
    for f in frames_list:
        frame = cv2.imread(f)
        image_bytes = cv2.imencode('.png', frame)[1].tobytes()
        response = rekognition.detect_custom_labels(
                        Image={'Bytes': image_bytes},
                        ProjectVersionArn = projectVersionArn
                    )
        if (len(response["CustomLabels"]) > 0):
            for elabel in response["CustomLabels"]:
                if int(elabel["Confidence"]) > 50:
                    left,top,right,bottom = transform_bounding(size ,elabel["Geometry"]["BoundingBox"])
                    label = elabel["Name"]
                    conf = elabel["Confidence"]

                    imgWidth, imgHeight= size
                    thick = int((imgHeight + imgWidth) // 500)

                    color = (0,255,0)
                    cv2.rectangle(frame,(left, top), (right, bottom), color, thick)
                    cv2.putText(frame, label+":"+str(conf)[0:4], (left, top - 12), 0, 1e-3 * imgHeight, color, thick//1)    
                    cv2.imwrite(f,frame)  
                else:
                    cv2.imwrite(f,frame)
        else:
            cv2.imwrite(f,frame)
            
def detect_labels(frames_queue):
    while True:
        try:
            task = frames_queue.get_nowait()
        except queue.Empty:
            print("Queue Empty")
            break
        else:
            process_frames(task)
    return True

def get_video_info(video):
    cap = cv2.VideoCapture(original_video)
    fps = cap.get(cv2.CAP_PROP_FPS)
    size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    cap.release()
    return fps, size

### Input/Output Configuration
Upload a video into the "input_video" folder to be processed. Next, specify the name of the file below.

In [ ]:
#---------------------------------------------------------------------------
# INSERT THE NAME OF THE INPUT VIDEO LOCATED IN THE INPUT_VIDEO FOLDER
original_video_name = "***.mp4" # eg. "original.mp4"
#---------------------------------------------------------------------------

In [ ]:
original_video = "input_video/{}".format(original_video_name)
frames_folder = "frames-{}".format(original_video_name.split('.')[0])
output_video = "output_video/{}-labeled.mp4".format(original_video_name.split('.')[0])
fps, size = get_video_info(original_video)

Review the video you have chosen to process.

In [ ]:
Video(original_video)

### Split the video into frames 

Now we are going to split our input video into frames, we'll use FFMPEG for this task and save the frames into a frames folder.

In [ ]:
print("Splitting frames...")
os.mkdir(frames_folder)
!ffmpeg -hide_banner -loglevel error -i {original_video} {frames_folder}/frame-%03d.png
print("Splitting frames... -- Complete!")

### Multiprocessing Configuration

Now we have our video split into frames let's move them into a queue divided in X chunks.

In [ ]:
files_list = glob.glob("{}/*".format(frames_folder))
n = 20 #Frames divided into chunks of n
number_of_chunks = int(len(files_list)/n)+1
split_list = list(chunks(files_list, number_of_chunks))

frames_queue = Queue()
for chunk in split_list:
    frames_queue.put(chunk)

number_of_processors = 5 #Number of subprocesses
processes = []

print("Size of queue:",frames_queue.qsize())

### Get bounding boxes for frames and overwrite the image file

Let's iterate over the queue of chunks (using multiprocessing) to call Amazon Rekognition Custom Labels to detect objects in our frames.

In [ ]:
print("Detecting Labels...")
for w in range(number_of_processors):
    p = Process(target=detect_labels, args=(frames_queue,))
    processes.append(p)
    p.start()
for p in processes:
        p.join()
        p.kill()
print("Detecting Labels... -- Complete!")

### Create the labeled video from frames

Once we have finished detecting objects and drawing the bounding boxes over the frames, we can stich the video back together.

In [ ]:
print("Creating output video...")
!ffmpeg -hide_banner -loglevel error -f image2 -r {fps} -i {frames_folder}/frame-%03d.png -vcodec libx264 -crf 18  -pix_fmt yuv420p {output_video} -y
print("Creating output video... -- Complete!")
shutil.rmtree(frames_folder)

Review your labeled video

In [ ]:
Video(output_video)